# Randomised Iterative Improvement on MAXSAT



Set classpath to the pre-compiled jar

In [1]:
@file:DependsOn("../build/libs/kglsm.jar")
@file:DependsOn("ch.qos.logback:logback-classic:1.2.3")
@file:DependsOn("ch.qos.logback:logback-core:1.2.3")
@file:DependsOn("io.github.microutils:kotlin-logging-jvm:2.0.3")

Define imports

In [2]:
import com.sihvi.glsm.problem.MAXSAT
import com.sihvi.glsm.sls.GLSMBuilder
import com.sihvi.glsm.sls.StateMachineTransition
import com.sihvi.glsm.space.BooleanSearchSpace
import com.sihvi.glsm.space.DiscreteSearchSpace
import com.sihvi.glsm.strategy.IIMode
import com.sihvi.glsm.strategy.IterativeImprovementStrategy
import com.sihvi.glsm.strategy.RandomWalkStrategy
import com.sihvi.glsm.transitionpredicate.NoImprovementPredicate
import com.sihvi.glsm.transitionpredicate.NotPredicate
import com.sihvi.glsm.transitionpredicate.ProbabilisticPredicate
import com.sihvi.glsm.memory.Memory
import com.sihvi.glsm.memory.BasicMemory
import com.sihvi.glsm.memory.BasicSolution

## Problem

We are using MAXSAT problem as an example. Get an instance of the problem.

In [3]:
val noVariables = 20

val problemInstance = MAXSAT.getRandomInstance(noVariables,50,2)
println(problemInstance)

(¬x6∨¬x0)∧(¬x5∨¬x17)∧(x5∨x7)∧(¬x1∨¬x0)∧(¬x16∨x4)∧(¬x0∨¬x7)∧(¬x3∨¬x6)∧(x2∨¬x12)∧(¬x13∨x11)∧(¬x19∨x7)∧(x19∨x4)∧(x16∨¬x8)∧(x6∨x0)∧(x12∨¬x3)∧(¬x9∨¬x1)∧(¬x19∨¬x18)∧(x10∨¬x5)∧(x5∨x17)∧(¬x16∨¬x12)∧(¬x11∨x13)∧(x16∨¬x10)∧(x9∨x17)∧(x12∨¬x4)∧(¬x5∨x0)∧(x18∨x15)∧(x12∨x18)∧(x12∨¬x17)∧(¬x1∨¬x5)∧(¬x1∨x19)∧(¬x14∨x9)∧(x4∨¬x6)∧(¬x14∨x18)∧(x13∨x1)∧(¬x18∨¬x4)∧(x11∨x2)∧(x15∨x6)∧(¬x10∨x13)∧(x18∨¬x18)∧(¬x12∨¬x19)∧(¬x4∨x1)∧(¬x14∨¬x4)∧(x19∨¬x12)∧(x4∨x14)∧(¬x10∨¬x17)∧(¬x5∨¬x10)∧(¬x12∨¬x7)∧(x5∨x18)∧(x19∨x8)∧(x3∨¬x7)∧(x11∨x7)


## Search space
Boolean search space defines two methods on an array of booleans: to get a neighbourhood and a random neighbour.

In this case, the members of the neighbourhood are exactly Hamming distance of 1 away from the current solution (i.e. one boolean value flipped) 

In [4]:
val space = BooleanSearchSpace(noVariables)

## Memory
Basic memory that holds:
* Current solution and its cost
* Best solution and best cost
* Number of steps performed
* Number of steps performed without improvement (when it was expected)

In [5]:
val initialSolution = space.getInitial()
val memory = BasicMemory(BasicSolution(initialSolution, problemInstance.evaluate(initialSolution)))

## GLSM and Strategies
Randomised Iterative Improvement consists of two strategies that are flipped probabilistically
* Iterative Best Improvement Strategy -- picks a solution from a neighbourhood that gives the best improvement
* Random Walk Strategy -- randomly picks a solution from a neighbourhood

The termination predicate of choice here is No Improvement Predicate, i.e. we terminate the search if there were n steps without improvement (in this case 10)

In [6]:
val terminationPredicate = NoImprovementPredicate(10)

val wp = 0.1
val toRandomPredicate = ProbabilisticPredicate(to = wp)
val toIIPredicate = NotPredicate(toRandomPredicate)

val walk = RandomWalkStrategy<Boolean>()
val iterativeImprovement = IterativeImprovementStrategy<Boolean>(IIMode.BEST, true)

As all the components are defined, we can now build the GLSM with strategies and transitions between them 

In [7]:
val glsm = GLSMBuilder<Boolean, BasicSolution<Boolean>, Memory<Boolean, BasicSolution<Boolean>>, DiscreteSearchSpace<Boolean>>()
        .addStrategy(iterativeImprovement)
        .addStrategy(walk)
        .addTransition(StateMachineTransition(0, 1, toRandomPredicate))
        .addTransition(StateMachineTransition(1, 0, toIIPredicate))
        .addTransition(StateMachineTransition(0, -1, terminationPredicate))
        .build()

In [8]:
glsm

digraph G {
S [label="Enter"]
-1 [label="Terminate", border="double"]
 0 [label="Best Iterative Improvement [updating best]"]
1 [label="Random Walk"]
0 -> 1 [label="X <= 0.1"]
1 -> 0 [label="¬(X <= 0.1)"]
0 -> -1 [label="NO_IMPROVEMENT >= 10"]
S -> 0 [label="⊤"]
}

In [9]:
glsm.toASCII()

                            +--------------------------------------------+
                            |                   Enter                    |
                            +--------------------------------------------+
                              |
                              | ⊤
                              v
+-------------+  X <= 0.1   +--------------------------------------------+
| Random Walk | <---------- | Best Iterative Improvement [updating best] | <+
+-------------+             +--------------------------------------------+  |
  |                           |                                             |
  |                           | NO_IMPROVEMENT >= 10                        |
  |                           v                                             |
  |                         #============================================#  | ¬(X <= 0.1)
  |                         H                 Terminate                  H  |
  |                         #===============

# Solve

With everything ready we can run GLSM on our problem instance

In [8]:
val finalSolution = glsm.solve(memory, space, problemInstance::evaluate)

println("Steps taken: " + memory.stepCount)
println("Solution: " + finalSolution.solution.joinToString(", "))
println("Cost: " + finalSolution.cost)

Steps taken: 18
Solution: false, true, true, true, true, false, true, false, true, false, true, false, false, false, false, true, true, false, true, true
Cost: 0.07999999999999996
